In [2]:
%matplotlib inline

import pyfits
import specutils
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from astropy.io import fits
import numpy as np
import sys
import time

import pandas as pd
#import seaborn as sns
from scipy import stats
#print(ea.__version__)
#import cPickle
import string

from astropy.table import Table
from __future__ import division

from astropy.modeling import models, fitting

from astropy.io import ascii

def plot_pretty(dpi=175, fontsize=9):
    # import pyplot and set some parameters to make plots prettier
    import matplotlib.pyplot as plt

    plt.rc("savefig", dpi=dpi)
    plt.rc('text', usetex=False)
    plt.rc('font', size=fontsize)
    plt.rc('xtick.major', pad=5)
    plt.rc('xtick.minor', pad=5)
    plt.rc('ytick.major', pad=5)
    plt.rc('ytick.minor', pad=5)
    return

plot_pretty()

### Part A: Dealing with Data

For more theory -- visit Part B (second half of this notebook)

In [3]:
# execute this line:
from astroquery.sdss import SDSS

/Users/Zoheyr/anaconda/lib/python2.7/site-packages/astroquery/sdss/__init__.py:28: UserWarning: Experimental: SDSS has not yet been refactored to have its API match the rest of astroquery (but it's nearly there).
  warnings.warn("Experimental: SDSS has not yet been refactored to have its API "


In [4]:
TSquery = """SELECT TOP 10000 
             p.psfMag_r, p.fiberMag_r, p.fiber2Mag_r, p.petroMag_r, 
             p.deVMag_r, p.expMag_r, p.modelMag_r, p.cModelMag_r, 
             s.class
             FROM PhotoObjAll AS p JOIN specObjAll s ON s.bestobjid = p.objid
             WHERE p.mode = 1 AND s.sciencePrimary = 1 AND p.clean = 1 AND s.class != 'QSO'
             ORDER BY p.objid ASC
               """
SDSSts = SDSS.query_sql(TSquery)
SDSSts


psfMag_r,fiberMag_r,fiber2Mag_r,petroMag_r,deVMag_r,expMag_r,modelMag_r,cModelMag_r,class
float64,float64,float64,float64,float64,float64,float64,float64,str6
18.50319,18.65275,19.33509,17.54539,17.31526,17.58132,17.58132,17.48715,GALAXY
19.02659,19.32441,19.80892,19.05827,19.03468,19.03111,19.03111,19.03111,STAR
19.8809,19.77895,20.46623,19.3534,19.1864,19.35493,19.35493,19.24559,GALAXY
22.03563,22.06141,22.68416,21.51795,21.03554,21.31751,21.31751,21.31751,GALAXY
21.56726,21.57312,22.22178,20.4583,19.93309,20.39825,20.39819,20.20402,GALAXY
18.66813,18.75309,19.36792,17.83372,17.69468,17.89914,17.6947,17.75269,GALAXY
20.19068,20.33947,20.93226,19.77666,19.63458,19.74873,19.74872,19.74873,GALAXY
19.41619,19.38348,20.15486,17.67687,17.16815,17.63962,17.63962,17.63962,GALAXY
18.88878,18.91179,19.57415,17.3317,17.02896,17.43539,17.02898,17.06256,GALAXY


#### Exercise 1: Visualize this data set. What representation is most appropriate, do you think?

#### Generating a dataset in the right format?

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# set the random state
rs = 23  # we are in Chicago after all

# extract feature names, remove class
feats = list(SDSSts.columns)
feats.remove('class')

# cast astropy table to pandas, remove classes
X = np.array(SDSSts[feats].to_pandas())

# our classes are the outcomes to classify on
y = np.array(SDSSts['class'])

# let's do a split in training and test set:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = rs)

## Cross-validating Multiple Model Components

In most machine learning applications, your machine learning algorithm might not be the only component having free parameters. You might not even be sure which machine learning algorithm to use! 

For demonstration purposes, imagine you have many features, but many of them might be correlated. A standard dimensionality reduction technique to use is [Principal Component Analysis](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). 

**Exercise 4**: The number of features in our present data set is pretty small, but let's nevertheless attempt to reduce dimensionality with PCA. Run a PCA decomposition in 2 dimensions and plot the results. Colour-code stars versus calaxies. How well do they separate along the principal components?

*Hint*: Think about whether you can run PCA on training and test set separately, or whether you need to run it on both together *before* doing the train-test split?

In [ ]:
from sklearn.decomposition import PCA

# instantiate the PCA object
pca = PCA(n_components=2)

# fit and transform the samples:
X_pca = pca.fit_transform(X)

# make a plot object
fig, ax = plt.subplots(1, 1, figsize=(12,8))

# loop over number of classes:
for i,l in enumerate(np.unique(y)):
    members = y == l
    plt.scatter(X_pca[members, 0], X_pca[members, 1], 
                color=sns.color_palette("colorblind",8)[i],
                label=l)
    
ax.set_xlabel("PCA Component 1")
ax.set_ylabel("PCA Component 2")
    
plt.legend()

But how do we know whether two components was really the right number to choose? perhaps it should have been three? Or four? Ideally, we would like to include the feature engineering in our cross validation procedure. In principle, you can do this by running a complicated for-loop. In practice, this is what `scikit-learn`s [Pipeline](http://scikit-learn.org/stable/modules/pipeline.html) is for! A `Pipeline` object takes a list of tuples of `("string", ScikitLearnObject)` pairs as input and strings them together (your feature vector `X` will be put first through the first object, then the second object and so on sequentially).

**Note**: `scikit-learn` distinguishes between *transformers* (i.e. classes that transform the features into something else, like PCA, t-SNE, StandardScaler, ...) and *predictors* (i.e. classes that produce predictions, such as random forests, logistic regression, ...). In a pipeline, all but the last objects must be transformers; the last object can be either.

**Exercise 6**: Make a pipeline including (1) a PCA object and (2) a random forest classifier. Cross-validate both the PCA components and the parameters of the random forest classifier. What is the best number of PCA components to use?

*Hint*: You can also use the convenience function [`make_pipeline`](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html#sklearn.pipeline.make_pipeline) to creatue your pipeline. 

*Hint*: Check the documentation for the precise notation to use for cross-validating parameters.

In [ ]:
from sklearn.pipeline import Pipeline

# make a list of name-estimator tuples
estimators = [('pca', PCA()), ('clf', RandomForestClassifier())]

# instantiate the pipeline
pipe = Pipeline(estimators)

# make a dictionary of parameters
params = dict(pca__n_components=[2, 4, 6, 8],
              clf__n_estimators=[10, 100, 300],
              clf__min_samples_leaf=[1,10])


##### Break

### Part B: A different approach, a different dataset

A consistent problem for visualizing large datasets is that almost all projections of the data must be rendered in two dimensions so they can be displayed on the page or screen. While it is possible to capture a greater deal of dimensionality (3D projection, point color, point size, point symbol, etc.) by getting a little creative, these methods are highly inadequate for data sets in 10s, 100s, or millions (Google) of dimensions.

A solution for this problem is dimesionality reduction. The goal is to identify a new set of artificial features that reduces the overall size of the data set while still capturing most of the variance of the original data (some instances require a similar, but different, goal of identifying which features contain the least information and removing those from the model).

#### Digits Data Set

There are many different algorithms that can be used to perform dimensionality reduction. Today we will introduce a few, and develop some intuition by example. To illustrate these examples, we will use the famous digits data set, which includes images of hand-written digits from 0-9. The images are segmented into an 8 x 8 [64] grid, with the intensity recorded in each pixel recorded on a scale from 0-16. *Note* - our data structure will deal with "flattened", length = 64, 1D arrays for each of the 1797 digits in the training set.

Historical aside - the use of machine learning to classify this famous data set has saved the USPS billions of dollars as computers can now scan and automatically direct > 98% of all mail based on the written zipcodes.

At face value, we do not have a good means for visualizing this 64 dimension data set, but dimensionality reduction can help us understand the structure of the data. Let us begin by loading and examining the data.

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
print(np.shape(digits.data))

As has already been noted - there are 64 pixels used to describe 1797 different characters. We can reshape the data to see how the digits actually appear:
Note - this example shows the first digit in the data set, feel free to examine others. If you cannot tell what digit you are looking at, you can access that via digits.target[digit_num].

In [ ]:
digit_num = 0

print(digits.data[digit_num])  # show the flattened array
grid_data = np.reshape(digits.data[digit_num], (8,8))  # reshape from 1d to 2d pixel array
plt.imshow(grid_data, interpolation = "nearest", cmap = "bone_r")
plt.colorbar()

#### Principal Component Analysis

Probably the most famous (not to be confused with the best) method of dimensionality reduction adopted throughout the astronomical literature is [Principal Component Analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) (PCA). We will skip the detailed mathematics, but in short, PCA aims to reduce the dimensionality by transforming the data set to a set of linearly uncorrelated variables called principle components. The transformation is accomplished following a singular value decomposition (SVD) of the data matrix, $X$, and in many instances most of the variance of the data is retained with only a few principle components. Advantages of PCA include that it is deterministic, and in many cases, fast. One of the main disadvantages is that PCA imposes linearity and orthogonality on the projected components, which may not be adequate to describe the underlying manifold of the data (as an aside - PCA was developed over a century ago, statisticians have crafted many alternatives in the intervening time).

I have determined that the two most important pixels for differentiating digits as (0-indexed) pixels 52 and 36.  (Note - we will discuss further how I determined this tomorrow.) 

**Problem 2a** Establish a baseline for the performance of our various dimensionality reduction techniques by plotting (0-indexed) pixel 52 against pixel 36 for the digits data set. Color the points by the digit they represent. 

*Hint - this is one application where the choice of color map **really** matters. I recommend `nipy_spectral`, and this is the rare case where `jet` is a good choice.*

In [ ]:
plt.scatter( digits.data[:, 52], 
             digits.data[:, 36], 
             c = digits.target, cmap = "nipy_spectral", edgecolor = "None")
plt.xlabel('pixel 52')
plt.ylabel('pixel 36')
plt.colorbar()

From this plot we see that just these two pixels do a decent job of clustering the data: the zeros are in the lower-right corner and the 7s are in the upper-right. However, there is a lot of confusion among the other digits, meaning these two pixels only capture a small fraction of the variance in the data. 

In an attempt to capture even more of the variance, we will now project the data using PCA. Again, brushing over the mathematical details, the PCA model projects the data with the following form:

$$x_i = \mu + \sum_{j = 1}^n a_{ij} v_j$$

where $x_i$ represents an individual digit image, $\mu$ represents the mean digit image, while the sum is over the eigenvectors (principal components) of the model $v_j$. The standard procedure is to list the most important eigenvectors with low $j$. In many cases, it is possible to capture most of the variance in the data using only the first few eigenvectors. 



We will project the data using the [`PCA`](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA) function within the [`sklearn.decomposition`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition) module. PCA models are fit in the following fashion:

    from sklearn.decomposition import PCA
    pca = PCA(n_components = None)
    pca.fit(X)
    
where `n_components` is the number of PCA components to retain (the default is the minimum between the number of samples and number of features), and `X` is the feature array.

**Problem 2b** Fit for the digits data set for the 2 leading PCA components. 

*Note - this isn't essential for this problem, but SVD on large matricies is computationally expensive $\mathcal{O}[N^3]$. For very large data sets [`RandomizedPCA`](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.RandomizedPCA.html) can significantly improve computation times, at the cost of some accuracy.*

In [ ]:
from sklearn.decomposition import PCA


pca = PCA(n_components = 2)
pca.fit(digits.data)

Now that we have fit the PCA model, we can transform the data using the `.transform()` method, which requires a feature array as input. 

**Problem 2c** Transform the digits data to the 2 leading PCA components, and make a scatter plot of the data in this projection. Color the points by their respective digits. How does this separation of the data compare to the use of just pixels 52 and 36?

In [ ]:
v_j = pca.transform(digits.data)

plt.scatter(v_j[:,0], v_j[:,1], c = digits.target, cmap = "nipy_spectral", edgecolor = "None")
plt.xlabel(r'$a_{i1}$', fontsize = 20)
plt.ylabel(r'$a_{i2}$', fontsize = 20)
plt.colorbar()

Here, we see that the PCA projection clusters (and visualizes) the data in a much more meaningful way.

The actual SVD eignvectors can be accessed via the `.components_` attribute, while the variance explained by each component is stored in the `.explained_variance_ratio_`.

**Problem 2d** Determine the variance explained by each of the components, as well as the total variance explained by the two components.

In [ ]:
print("The first component explains {:.3f} of the variance in the data.".format(pca.explained_variance_ratio_[0]))
print("The second component explains {:.3f} of the variance in the data.".format(pca.explained_variance_ratio_[1]))
print("The top 2 components explain {:.3f} of the variance in the data.".format(sum(pca.explained_variance_ratio_)))

**Problem 2e** How many components are needed to explain 90% of the variance in the data.

In [ ]:
for num_feats in np.arange(1,65, dtype = int):
    pca = PCA(n_components = num_feats)
    pca.fit(digits.data)
    variance_explained = sum(pca.explained_variance_ratio_)
    if variance_explained >= .90:
        break
print("{:d} features are needed to explain 90% of the variance".format(num_feats))

We started with 64 features to describe the digits dataset, and now we are able to project those features into a 21-D space that captures 90% of the original variance. Doing so has allowed us to readily separate the different characters via a 2-D projection, which makes it easy to see the difference between the different numbers. 

Dimensionality reduction is useful beyond the visualization of high-dimensional data sets. It is often used as a preprocessing step before passing data to machine-learning models (imagine Google with $10^6$ features - if this can be reduced to $\sim{10}\mathrm{s}$ it will greatly improve the run time of the algorithms. PCA in particular removes correlation between the features, which is particularly useful for some algorithms, like Support Vector Machines. Finally, for a method like PCA, the eigenvectors can provide insight into the meaning of the principal components. Once again, our very own Jake VanderPlas has put together a nice tutorial on [extracting and visualing the eigenvectors of SDSS galaxy spectra](http://www.astroml.org/sklearn_tutorial/dimensionality_reduction.html). A brief note of caution - astronomers will often try to derive physical insight from PCA eigenspectra or eigentimeseries, but this is not advisable as there is no physical reason for the data to be linearly and orthogonally separable.

###Manifold Learning

*Why the advised caution against PCA?* As already noted, newer, more flexible methods have been developed. Furthermore, it is possible that the data of interest have interesting non-linear structure, which is not easily identified via PCA-like projections. Thus, an entire field, known as [Manifold Learning](http://scikit-learn.org/stable/modules/manifold.html), has been developed, with several algorithms available via the [`sklearn.manifold`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.manifold) module. 

Aside: I will note that an advantage of PCA is its deterministic nature, leading to stable repeatability. The results from most non-linear manifold-learning methods change from run to run.

